## Day 21: Trench Map

[![nbviewer](https://raw.githubusercontent.com/jupyter/design/master/logos/Badges/nbviewer_badge.svg)](https://nbviewer.org/github/mazharenko/AoC-2021/tree/HEAD/notebooks/day21/puzzle.ipynb)

[Problem statement](https://adventofcode.com/2021/day/21)

### Part 2

In [ ]:
type PlayerState =
    { Position: int
      Score: int }
    member this.Move(steps) =
        let newPos = (this.Position + steps - 1) % 10 + 1

        { Position = newPos
          Score = this.Score + newPos }

type Die =
    { Rolled: int }
    // rolling quantum d3 three times is the
    // same as rolling quantum d27 once.
    static member Split() =
        [ for i in [ 1 .. 3 ] do
              for j in [ 1 .. 3 ] do
                  for k in [ 1 .. 3 ] do
                      yield { Rolled = i + j + k } ]
                      
type GameState = { Players: PlayerState*PlayerState }

type WinStat = WinStat of int64*int64 with
    static member (+)
        (WinStat (stat1p1, stat1p2), WinStat (stat2p1, stat2p2)) =
        WinStat (stat1p1 + stat2p1, stat1p2 + stat2p2)
    static member get_Zero() = WinStat (0L, 0L)
    
let swap (WinStat (p1, p2)) =
    WinStat (p2, p1)

In [ ]:
open System.Collections.Generic

let memoize f =
    let cache = Dictionary<_, _>()    
    fun x ->
        match cache.TryGetValue(x) with
        | true, value -> value
        | _ ->
            let value = f x
            cache.Add(x, value)
            value

In [ ]:
let sampleGame = {
    Players = 
        {Position = 4; Score = 0;},
        {Position = 8; Score = 0;}
}

let actualGame = {
    Players =
        {Position = 6; Score = 0;},
        {Position = 9; Score = 0;}
}

In [ ]:

#nowarn "40" 
let rec play =
    memoize (fun state ->        
        let (playerCurrent, playerNext) = state.Players

        if (playerCurrent.Score >= 21) then
            WinStat (1L, 0L)
        elif (playerNext.Score >= 21) then
            WinStat (0L, 1L)
        else
            let dice = Die.Split()
            dice
            |> List.map (fun die ->
                play {
                    Players = ( // swap players
                        playerNext,
                        playerCurrent.Move die.Rolled
                    )
                }
            )
            |> List.map swap // and so swap results back
            |> List.sum
)

In [ ]:
sampleGame |> play |> display
actualGame |> play |> display

Item1,Item2
444356092776315,341960390180808


Item1,Item2
486638407378784,413013330504401
